In [ ]:
import process_copus
import numpy as np
import glob
import os
import matplotlib
import matplotlib.pyplot as plt
import scipy.io
%matplotlib inline

In [ ]:
nb_id = 1734373836
import time
try:
    print(nb_id)
except NameError:
    print(round(time.time()))

In [ ]:
def get_txt_filename(path_raw):
    path = os.path.abspath(path_raw)
    root, ext = os.path.splitext(path)
    txt_filename = root + '.txt'
    return txt_filename

In [ ]:
def out_path(filename):
    global outdir
    path = os.path.join(outdir, filename)
    return path

In [ ]:
def get_filename_prefix(filepath):
    filename = os.path.basename(filepath)
    root, ext = os.path.splitext(filename)
    return root

In [ ]:
def get_outdir(filepath_raw):
    filepath = os.path.abspath(filepath_raw)
    head1, tail1 = os.path.split(filepath)
    head2, tail2 = os.path.split(head1)
    return os.path.join(tail2, tail1)

In [ ]:
def get_parent_dir(filepath_raw):
    filepath = os.path.abspath(filepath_raw)
    head1, tail1 = os.path.split(filepath)
    head2, tail2 = os.path.split(head1)
    return tail2

In [ ]:
def get_topdir_base(filepath_raw):
    filepath = os.path.abspath(filepath_raw)
    head1, tail1 = os.path.split(filepath)
    root, ext = os.path.splitext(tail1)
    return root

In [ ]:
dirpath = '/scratch/n.beaver/14_copus_isofreq_permalloy/copus_isofreq_permalloy_000.out/'
outdir = get_outdir(dirpath)

In [ ]:
get_topdir_base(dirpath)

In [ ]:
# os.listdir(dirpath)

In [ ]:
npy_filename = 'm_full000184.npy'
npy_filepath = os.path.join(dirpath, npy_filename)
M1_raw = np.load(npy_filepath)
filename_prefix = get_filename_prefix(npy_filepath)
title_prefix = filename_prefix


In [ ]:
if not os.path.isdir(outdir):
    os.makedirs(outdir, exist_ok=True)

In [ ]:
txt_path = get_txt_filename(dirpath)
with open(txt_path) as fp:
    params = process_copus.parse_logfile(fp.readlines())

In [ ]:
GHz = 1e9
ps = 1e-12
nm = 1e-9
um = 1e-6
subtitle_prefix = "f = {:.1f} GHz, dt = {} ps".format(params['f']/GHz, params['tstep']/ps)

In [ ]:
M1_raw.shape

In [ ]:
params

In [ ]:
Nx = int(params['Nx'])
Ny = int(params['Ny'])
dx = params['c']
dy = params['c']
xpos = np.linspace(0, Nx, Nx)*dx
ypos = np.linspace(0, Ny, Ny)*dy

In [ ]:
xpos/um

In [ ]:
kx = 2*np.pi*np.fft.fftshift(np.fft.fftfreq(Nx, d=dx))
ky = 2*np.pi*np.fft.fftshift(np.fft.fftfreq(Ny, d=dx))

In [ ]:
M1 = M1_raw.squeeze()
M1z = M1[2]

In [ ]:
M1.shape

In [ ]:
M1_fftz_complex_unshifted = np.fft.fft2(M1z)
M1_fftz_complex = np.fft.fftshift(M1_fftz_complex_unshifted)
M1_fftz = np.abs(M1_fftz_complex)

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
ax.hist(M1z.flatten());
ax.set_yscale('log');
ax.set_ylabel('bin count')
ax.set_xlabel("$M_z$ [A/m]")
ax.set_title(subtitle_prefix + ", nb_id = {}".format(nb_id) + "\n" + outdir, fontsize=9)
fig.suptitle("$M_z$, " + title_prefix);

In [ ]:
fig.savefig(out_path(filename_prefix + "_histogram_M1z.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
M1z[2].min(), M1z[2].max()

In [ ]:
xpos.shape, ypos.shape, M1z.shape

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(2*6.4, 2*4.8)
)
norm = matplotlib.colors.TwoSlopeNorm(vmin=-13, vcenter=0, vmax=13)
plot = ax.pcolormesh(
    xpos/um,
    ypos/um,
    M1z,
    cmap='bwr',
#     vmin=-20,
#     vmax=-100,
    norm = norm,
    shading='nearest',
#     norm=matplotlib.colors.LogNorm(),
);
ax.set_xlabel('x position [um]')
ax.set_ylabel('y position [um]')
fig.colorbar(mappable=plot, ax=ax, label="$M_z$ [A/m]")
ax.set_aspect('equal')
ax.set_title(subtitle_prefix + ", nb_id = {}".format(nb_id) + "\n" + outdir, fontsize=9)
fig.suptitle("$M_z$, " + title_prefix);

In [ ]:
fig.savefig(out_path(filename_prefix + "_heatmap_M1z.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(2*6.4, 2*4.8)
)
blackwhite_cm = matplotlib.colors.LinearSegmentedColormap.from_list('black_white', ['black', 'white'], N=2)
plot = ax.pcolormesh(
    xpos/um,
    ypos/um,
    np.sign(M1z),
    cmap=blackwhite_cm,
    shading='nearest',
);
ax.set_xlabel('x position [um]')
ax.set_ylabel('y position [um]')
# fig.colorbar(mappable=plot, ax=ax, label="$M_z$ [A/m]")
ax.set_aspect('equal')
ax.set_title(subtitle_prefix + ", nb_id = {}".format(nb_id) + "\n" + outdir, fontsize=9)
fig.suptitle("$M_z$, " + title_prefix);

In [ ]:
fig.savefig(out_path(filename_prefix + "_heatmap_M1z_bw.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(2*6.4, 2*4.8)
)
plot = ax.pcolormesh(
    xpos/um,
    ypos/um,
    M1z,
    cmap='magma',
    norm=matplotlib.colors.LogNorm(),
    shading='nearest',
);
ax.set_xlabel('x position [um]')
ax.set_ylabel('y position [um]')
fig.colorbar(mappable=plot, ax=ax, label="$M_z$ [A/m]")
ax.set_aspect('equal')
ax.set_title(subtitle_prefix + ", nb_id = {}".format(nb_id) + "\n" + outdir, fontsize=9)
fig.suptitle("$M_z$, " + title_prefix);

In [ ]:
fig.savefig(out_path(filename_prefix + "_heatmap_M1z_log.png"), bbox_inches='tight');

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
M1_fftz.min(), M1_fftz.max()

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(2*6.4, 2*4.8),
)
plot = ax.pcolormesh(
    kx*um,
    ky*um,
    M1_fftz,
    cmap='magma',
#     norm=matplotlib.colors.LogNorm(),
    shading='nearest',
);
ax.set_xlabel('$k_x$ [1/um]')
ax.set_ylabel('$k_y$ [1/um]')
fig.colorbar(mappable=plot, ax=ax, label="FFT magnitude for $M_z$")
ax.set_aspect('equal')
ax.set_title(subtitle_prefix + ", nb_id = {}".format(nb_id) + "\n" + outdir, fontsize=9)
fig.suptitle("2D FFT of $M_z$, " + title_prefix);

In [ ]:
fig.savefig(out_path(filename_prefix + "_heatmap_fft_M1z.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
M1_fftz

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(2*6.4, 2*4.8)
)
plot = ax.pcolormesh(
    kx*um,
    ky*um,
    M1_fftz,
    cmap='magma',
    norm=matplotlib.colors.LogNorm(),
    shading='nearest',
);
ax.set_xlabel('$k_x$ [1/um]')
ax.set_ylabel('$k_y$ [1/um]')
fig.colorbar(mappable=plot, ax=ax, label="FFT mag for $M_z$")
ax.set_aspect('equal')
ax.set_title(subtitle_prefix + ", nb_id = {}".format(nb_id) + "\n" + outdir, fontsize=9)
fig.suptitle("2D FFT of $M_z$, " + title_prefix);

In [ ]:
fig.savefig(out_path(filename_prefix + "_heatmap_fft_M1z_log.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
M1[1].min(), M1[1].max()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
norm = matplotlib.colors.TwoSlopeNorm(vcenter=0)
plot = ax.pcolormesh(
    xpos/um,
    ypos/um,
    M1[1],
    cmap='bwr',
    norm = norm,
    shading='nearest',
);
ax.set_xlabel('x position [um]')
ax.set_ylabel('y position [um]')
fig.colorbar(mappable=plot, ax=ax, label="$M_y$ [A/m]")
ax.set_aspect('equal')
ax.set_title(subtitle_prefix + ", nb_id = {}".format(nb_id) + "\n" + outdir, fontsize=9)
fig.suptitle("$M_y$, " + title_prefix);

In [ ]:
fig.savefig(out_path(filename_prefix + "_heatmap_M1y.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
M1[0].min(), M1[0].max()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
plot = ax.pcolormesh(
    xpos/um,
    ypos/um,
    M1[0],
    cmap='magma',
    shading='nearest',
#     norm=matplotlib.colors.LogNorm(),
);
ax.set_xlabel('x position [um]')
ax.set_ylabel('y position [um]')
fig.colorbar(mappable=plot, ax=ax, label="$M_x$ [A/m]")
ax.set_aspect('equal')
ax.set_title(subtitle_prefix + ", nb_id = {}".format(nb_id) + "\n" + outdir, fontsize=9)
fig.suptitle("$M_x$, " + title_prefix);

In [ ]:
fig.savefig(out_path(filename_prefix + "_heatmap_M1x.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
import scipy
to_save = {}
to_save.update(params)
to_save.update({
    'dirpath': dirpath,
    'npy_filename': npy_filename,
    'xpos': xpos,
    'ypos': ypos,
    'Mx': M1[0],
    'My': M1[1],
    'Mz': M1[2],
    'kx': kx,
    'ky': ky,
    'Mz_fft': M1_fftz,
    'outdir': outdir,
    'filename_prefix': filename_prefix,
    'title_prefix': title_prefix,
    'subtitle_prefix': subtitle_prefix,
    'nb_id': nb_id,
})

scipy.io.savemat(
    os.path.join(get_parent_dir(dirpath), get_topdir_base(dirpath) + '_with_fft.mat'),
    to_save,
    long_field_names=True,
    do_compression=True,
)